In [1]:
import eikon as ek
import pandas as pd
import numpy as np
import time
import datetime
import matplotlib.pyplot as plt
import os
import sqlite3
from sqlite3 import Error
from scipy import stats
import scipy as sp
import statsmodels.api as sm
import pylab
import statistics
from statsmodels.tsa.stattools import adfuller
from openpyxl import load_workbook
from openpyxl import Workbook
from datetime import date
from OLS import OLS

ek.set_app_key('ed89d084197d408686e3cfa56e3356daad46c055')

print('Done')

ticker = ['BTC=','LTC=','ETH=', 'BCH=', 'XRP=','EUR=','CHF=','GBP=','JPY=','CNY=','CAD=', 'AUD=','MXN=','CHFEUR=R', 'GBPEUR=R', 'JPYEUR=R','CNYEUR=R','CADEUR=R','AUDEUR=R','MXNEUR=R'] #wäre gut, wenn man sie nicht manuell eingeben muss, sondern aus einer Liste ziehen.
print('Retrieving Data for',ticker)

df1 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2019-01-01', end_date = '2019-04-11')
df2 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2019-04-12', end_date = '2019-09-08')
df3 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2019-09-09', end_date = '2020-02-05')
df4 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2020-02-06', end_date = '2020-07-04')
df5 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2020-07-05', end_date = '2020-12-01')
df6 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2020-12-02' , end_date = '2021-04-30')

df = pd.concat([df1,df2,df3,df4,df5,df6], axis = 0)

df.columns = df.columns.str.replace('=', '')

#Save data to an excel sheet
df.to_excel('AQM_Data.xlsx')
print('Retrieving Data successful.')


df_p = pd.read_excel('AQM_Data.xlsx')
df_p = df_p.set_index(df_p['Date'])
df_p = df_p.drop(axis=1, columns = 'Date')

#Cleaning Python Data

df_clean=pd.DataFrame(df_p)
df_clean.dropna(axis='rows',inplace=True)

print('table size before cleaning:', df.shape)
print('table size after cleaning:', df_clean.shape)

df_p = df_clean
df = df_p

#Read & clean excel Data

df_e = pd.read_excel('AQM_task_3_v3.xlsx')
df_e = df_e.set_index(df_e['Date'])
df_e = df_e.drop(axis=1, columns = 'Date')

df_clean2=pd.DataFrame(df_e)
df_clean2.dropna(axis='rows',inplace=True)

print('table size before cleaning:', df.shape)
print('table size after cleaning:', df_clean.shape)

df_e = df_clean2

#Check if the values are the same

ticker_new = [s.replace("=", "") for s in ticker]

for i in ticker_new:
    s = df_e [i].sum() - df_p[i].sum()
    m = df_e [i].mean() - df_p[i].mean()
    min = df_e [i].min() - df_p[i].min()
    max = df_e [i].max() - df_p[i].max()
    std = df_e [i].std() - df_p[i].std()
    if (s + m + min + max + std) == 0:
        print(i,': Data are equal.')        
    else:
        print (i,': Data are NOT equal.')
        
        
#Create a folder for the Charts, Histograms etc.

def create_folder(directory):
    try:
        if os.path.exists(directory):
            print("Folder '%s' already exists." %directory, "\nFiles will be stored in '%s'." %directory)
        if not os.path.exists(directory):
            os.makedirs(directory)
            print("Folder '%s' has been created." %directory, "\nFiles will be stored in '%s'." %directory)
    except OSError:
        print("Creation of the folder failed.")
        
create_folder('Charts')

def plots_save():
    for col in df.iteritems():
        print('Plotting '+col[0])
        fig, ax = plt.subplots()
        indicator= df[col[0]]
        ax.plot(indicator, alpha=0.9, color='blue')
        plt.title(col[0])
        plt.savefig('Charts\Plot_'+col[0]+'.png')
        plt.close()
        
plots_save()

#Check if values are missing 


ticker_new = [s.replace("=", "") for s in ticker]

for i in ticker_new:
    print('')
    if (df_e[i].isnull().sum().any() == False) and (df_p[i].isnull().sum().any() == False):
        print(i,': There isn\'t any wrong or missing data.')
        print('No cleaning needed.')
    else:
        if df_e[i].isnull().sum().any() == True:          
            print(i,': There is',df_e[i].isnull().sum(),'wrong or missing data in the Excel data!')

            print('Table size before cleaning:', df_e.shape)
            df_e.dropna(axis = 'rows', how = 'any', inplace = True)

            print('Table size after cleaning:', df_e.shape)
        else:          
            print(i,': There is',df_p[i].isnull().sum(),'wrong or missing data in the Python data!')

            print('Table size before cleaning:', df_p.shape)
            df_p.dropna(axis = 'rows', how = 'any', inplace = True)

            print('Table size after cleaning:', df_p.shape)

print('\nSince the data is identical, we can work only with the data from python (df_p = df)')


#Initial Analysis

df.describe()



Done
Retrieving Data for ['BTC=', 'LTC=', 'ETH=', 'BCH=', 'XRP=', 'EUR=', 'CHF=', 'GBP=', 'JPY=', 'CNY=', 'CAD=', 'AUD=', 'MXN=', 'CHFEUR=R', 'GBPEUR=R', 'JPYEUR=R', 'CNYEUR=R', 'CADEUR=R', 'AUDEUR=R', 'MXNEUR=R']
Retrieving Data successful.
table size before cleaning: (851, 20)
table size after cleaning: (565, 20)
table size before cleaning: (565, 20)
table size after cleaning: (565, 20)
BTC : Data are equal.
LTC : Data are equal.
ETH : Data are equal.
BCH : Data are equal.
XRP : Data are equal.
EUR : Data are equal.
CHF : Data are equal.
GBP : Data are equal.
JPY : Data are equal.
CNY : Data are equal.
CAD : Data are equal.
AUD : Data are equal.
MXN : Data are equal.
CHFEURR : Data are equal.
GBPEURR : Data are equal.
JPYEURR : Data are equal.
CNYEURR : Data are equal.
CADEURR : Data are equal.
AUDEURR : Data are equal.
MXNEURR : Data are equal.
Folder 'Charts' already exists. 
Files will be stored in 'Charts'.
Plotting BTC
Plotting LTC
Plotting ETH
Plotting BCH
Plotting XRP
Plotting

,BTC,LTC,ETH,BCH,XRP,EUR,CHF,GBP,JPY,CNY,CAD,AUD,MXN,CHFEURR,GBPEURR,JPYEURR,CNYEURR,CADEURR,AUDEURR,MXNEURR
count,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000
mean,15629.870973,84.351451,481.018283,322.428407,0.333371,1.140930,0.955103,1.294249,107.575575,6.851682,1.323259,0.703385,20.435345,0.918645,1.134350,0.815404,0.128008,0.662955,0.615983,0.043141
std,14807.965308,53.615460,572.003027,140.279803,0.224645,0.042767,0.039161,0.051021,2.200117,0.218781,0.039714,0.041151,1.585081,0.018571,0.029448,0.024195,0.002503,0.017175,0.018670,0.003627
min,3702.480000,31.950000,109.310000,122.910000,0.138380,1.069000,0.878300,1.148500,102.340000,6.425000,1.227900,0.574100,18.533400,0.872700,1.063700,0.757200,0.120410,0.628600,0.528600,0.036730
25%,8044.000000,47.350000,176.090000,233.140000,0.226690,1.107800,0.915300,1.254900,105.940000,6.703200,1.306000,0.679900,19.150500,0.906600,1.109600,0.796600,0.126510,0.647100,0.610300,0.039920
50%,9663.100000,61.040000,232.880000,284.630000,0.273910,1.124300,0.967900,1.293700,107.760000,6.907800,1.323500,0.697600,19.890000,0.922600,1.128700,0.817700,0.128060,0.662900,0.616500,0.041750
75%,12218.250000,94.150000,401.350000,375.810000,0.329730,1.181600,0.989400,1.320200,109.000000,7.038900,1.340100,0.725500,21.436000,0.930700,1.159800,0.831700,0.129630,0.680300,0.624800,0.046690
max,63398.460000,309.140000,2765.210000,1061.230000,1.824600,1.232500,1.020300,1.413900,112.170000,7.178600,1.451000,0.796700,25.338000,0.951300,1.204300,0.872400,0.133330,0.699800,0.654900,0.049830
